In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
np.seterr(over='raise');

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from helpers import *
DATA_TRAIN_PATH = '../train.csv' # TODO: download train data and supply path here 
y, X, ids = load_csv_data(DATA_TRAIN_PATH)

In [81]:
from preprocessing import *
# preprocess data
tX = preprocess_inputs(X)

In [93]:
from toolbox import *
# Define the parameters of the algorithm.
max_iters = 3000
gamma = 0.000005
lambda_ = 25
batch_size = 100
k = 4
k_indices = build_k_indices(y, k, 1)

loss_tr = []
loss_te = []

for k_ in range(k):
    
    test_indices = k_indices[k_]
    test_y, test_x = (y[test_indices], tX[test_indices])
    
    training_indices = np.ravel(np.delete(k_indices, k_, axis=0))
    training_y, training_x = (y[training_indices], tX[training_indices])
    
    w_star = reg_logistic_regression(training_y, training_x, lambda_, gamma, max_iters)
    
    loss_tr.append(calculate_loss_log_likelihood(training_y, training_x, w_star))
    loss_te.append(calculate_loss_log_likelihood(test_y, test_x, w_star))
        
print("Training error: {tr}\nTest error: {te}".format(tr=np.mean(loss_tr), te=np.mean(loss_te)))

Current iteration=0, the loss=129965.09635498974
Current iteration=100, the loss=-10321121686.590923
Current iteration=200, the loss=-10662185557.344978
Current iteration=300, the loss=-10899801333.320663
Current iteration=400, the loss=-11071298446.424082
Current iteration=500, the loss=-11196904827.619644
Current iteration=600, the loss=-11289929649.186924
Current iteration=700, the loss=-11359493394.944494
Current iteration=800, the loss=-11411996894.58586
Current iteration=900, the loss=-11452009454.978436
Current iteration=1000, the loss=-11482832185.000849
Current iteration=1100, the loss=-11506869584.279968
Current iteration=1200, the loss=-11525882702.74843
Current iteration=1300, the loss=-11541165982.769478
Current iteration=1400, the loss=-11553673089.767635
Current iteration=1500, the loss=-11564107629.622704
Current iteration=1600, the loss=-11572989149.197577
Current iteration=1700, the loss=-11580701451.170324
Current iteration=1800, the loss=-11587528106.962088
Current 

### Result analysis

#### experiment 1
- max_iters = 3000
- gamma = 0.0000025
- lambda_ = 500

Training error: -1120068718.9752452

Test error: -124086547.00130773

#### experiment 2
Binarize -999 by -1 instead of zero
- max_iters = 3000
- gamma = 0.0000025
- lambda_ = 500

Training error: -1120068718.9752452

Test error: -124086547.00130773

Does not seem to change anything to the computation

#### experiment 3
Remove standardization from course and 0 to 1 binarization for -999 values
- max_iters = 3000
- gamma = 0.00001
- lambda_ = 500

Training error: 36759800214.95009

Test error: 4107345854.1234426

#### experiment 4
-1 to 1 binarization for -999 values
- max_iters = 3000
- gamma = 0.000005
- lambda_ = 500

Training error: -8683001618.57466

Test error: -964334808.2575201

This seems to improve the classification quite significantly

#### experiment 5
-1 to 1 for discrete features
- max_iters = 3000
- gamma = 0.000005
- lambda_ = 500

Training error: -10516946495.735361

Test error: -1168079071.61974

This also improves the test error by quite a margin

#### experiment 6
Remove feature scaling from exponential family distributions
- max_iters = 3000
- gamma = 0.000005
- lambda_ = 500

Training error: 430048088.1856251

Test error: 48278445.35947897

This destroys the performance of the classifier


#### experiment 7
lambda modification
- max_iters = 3000
- gamma = 0.000005
- lambda_ = 250

Training error: -10840473622.745983

Test error: -1203987023.1683881

Still an improvement over experiment 5, should explore setting the lambda to even lower values

#### experiment 8
lambda modification
- max_iters = 3000
- gamma = 0.000005
- lambda_ = 50

Training error: -11435642570.154247

Test error: -1270057446.4032438

#### experiment 9
lambda modification
- max_iters = 3000
- gamma = 0.000005
- lambda_ = 1

Training error: -11267049837.002642

Test error: -1251301717.207404

Did not improve, the best lambda is probably between 50 and 1

## experiment 10 BEST
lambda modification
- max_iters = 3000
- gamma = 0.000005
- lambda_ = 25

Training error: -11452909733.500443

Test error: -1271963954.567976

Improved, should test lambda=32 and 12

In [94]:
if -1271963954.567976 < -1270057446.4032438:
    print("YES")
203987023.1683881-1270057446.4032438

YES


-1066070423.2348557

In [37]:
w = reg_logistic_regression(y, tX, lambda_, gamma, max_iters)

Current iteration=0, the loss=173286.79513998624
Current iteration=1000, the loss=-23667471819.96783
Current iteration=2000, the loss=-23738787341.081955
The loss=-23741795523.01027


## Generate predictions and save ouput in csv format for submission:

In [28]:
DATA_TEST_PATH = '../test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_te = preprocess_inputs(tX_test)

In [35]:
OUTPUT_PATH = '../submission.csv' # TODO: fill in desired name of output file for submission
y_pred = sigmoid(tX_te @ w)
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = -1
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [34]:
y_pred[y_pred > 0.5].shape

(358878,)